# Owl
## A Retirement Planning Laboratory

This package is a retirement modeling framework for exploring the sensitivity of retirement financial decisions. Strictly speaking, it is not a planning tool, but more an environment for exploring *what if* scenarios. It provides different realizations of a financial strategy. One can certainly have a savings plan, but due to the volatility of financial investments, it is impossible to have a certain asset earnings plan. This does not mean one cannot make decisions. These decisions need to be guided with an understanding of the sensitivity of the parameters.This is exactly where this tool fits it. Given your savings and spending desires, it can generate different future realizations of your strategy under different market assumptions, helping to better understand your financial situation.

The algorithm in Owl is using the open-source HiGHS linear programming solver.
The complete formulation and detailed description of the undelying
mathematical model can be found
[here](https://raw.githubusercontent.com/mdlacasse/Owl/main/docs/owl.pdf).

Copyright - Martin-D. Lacasse (2024)

Disclaimers: *I am not a financial planner. You make your own decisions. This program comes with no guarantee. Use at your own risk.*

### <span style="color: blue"> README FIRST </span>
<span style="color:black;background:yellow;font-weight:bold">
Don't make changes directly to this file. Keep it as a working example. Therefore, it is recommended that you build your own case by making a copy of the template files provided where you will be able to enter your own numbers and explore your own assumptions.
</span>

# Tutorial - Using the case of Jack and Jill

## Introduction
This file is provided as an example to introduce you to Owl.

It is assumed that you have some familiarity with using a jupyter notebook or jupyterLab, and some basic programming skills in Python. If not, a simple tutorial can guide you to the basic skills needed.

For simulating your own realizations, use the files beginning with *template*. Make a copy and rename them keeping the same extension and give them your own names. Then you'll be able to personalize a case with your own numbers and start experimenting with Owl.

This notebook named *jack+jill* describes the case of Jack and Jill, a ficticious couple used for demonstration.

### Just some Python module bookkeeping
This command needs to be at the beginning of every Owl notebook.

In [ ]:
import owl

## Initializing the life parameters for the realization
In order to be able to generate a realization of the future, one must start with providing the year of birth of each spouse(s) and their expected lifespan(s).

For selecting your own numbers, there are plenty of longevity predictors on the web. Pick your favorite:

https://longevityillustrator.org

https://www.livingto100.com/calculator

https://www.sunlife.ca/en/tools-and-resources/tools-and-calculators/life-expectancy-calculator/

or just Google life expectancy calculator.

There are two values needed for couples. Single individuals just enter one value in each list between square brackets `[ ]`. For couples, always keep the same order in the pair of values when entering the data.

When first creating a plan, the default values used for assets allocation ratios and rates of return will be reported. These values are listed to make you aware that these values would be used if no other choice is entered. We will cover how to do this in the next sections.

Here Jack was born in 1962 and Jill in 1965. Jack hopes to live to 89 years old, while Jill thinks she might reach age 92. Let's give this case a name.

In [ ]:
plan = owl.Plan([1962, 1965], [89, 92], 'jack & jill - tutorial')

## Specify account balances and spousal beneficiaries
For each spouse, savings accounts have three buckets comprising of the total value of:
- Individual **taxable** investment or savings accounts, including bank accounts, and CDs - do not include your safety net account which should typically be sufficient for sustaining 6 months of living expenses;
- **Tax-deferred** savings accounts, including all IRAs, 401k, 403b, etc.;
- **Tax-free** savings accounts, including Roth IRAs and Roth 401k.

For married couples, each spouse will have to enter values for each type of savings account, following the same order as before. For single individuals, only one value is needed between square brackets `[ ]`.

Most investment accounts have named beneficiaries. The *beneficiary* values specify the fraction of total of assets left to the other spouse at death. For example, a spouse leaving 3/4 of her fortune to her three children and the other part to her partner would have a beneficiary value of 0.25. While this number is irrelevant for single individuals, it still needs to be entered: just use `[1]`.

Jack has \\$90 k in his taxable account, \\$600 k in his 401k, and $50 k in his Roth 401k and \\$20 k in Roth IRAs. Jill has \\$60 k in her savings bank account, \\$150 k in a 403b, and \\$40 k in Roth IRAs in which she contributed over the years. Units for \\$ are in thousands by default. You can use the `units` optional keyword to indicate that the numbers are entered in other units, such as dollars (`1`) or millions (`M`). Notice that being in Python, arithmetics can be used while entering numbers.

In [ ]:
plan.setAccountBalances(
    taxable=[90, 60],
    taxDeferred=[600, 150],
    taxFree=[50 + 20, 40],
)

Jack is anticipated to pass first and leaves everything his spouse for all his savings account. This benificiary fraction is entered as [1, 1, 1], i.e., one for each type of account. Changing these results while optimizing for maximum bequest can lead to very unanticipated results.

In [ ]:
plan.setBeneficiaryFraction([1, 1, 1])

## There must be a plan for wages, savings, and Roth conversions
The most manageable part of retirement planning is the control one has over work income, contributions to savings accounts, Roth conversions, and other big spending items in the near- and mid-term future.
In order to execute a realization, one must provide an earning, saving, and Roth conversion plan. This is done through providing an Excel workbook with one spreadsheet (tab) per spouse with the following information:

|year|anticipated wages|ctrb taxable | ctrb 401k | ctrb Roth 401k | ctrb IRA | ctrb Roth IRA | Roth X | big ticket items|
|--|--|--|--|--|--|--|--|--|
|2024 | | | | | | | | |
|2025 | | | | | | | | |
| ... | | | | | | | | |
|20XX | | | | | | | | |

Here, 20XX is the last row which could be the last year based on the life expectancy values provided. Missing years will be filled with empty values. For the columns, *anticipated wages* is the annual amount (gross minus tax-deferred contributions) that you anticipate to receive from employment or other sources (not including dividends from your taxable investment accounts). Note that column names are case sensitive and all of these entries must be in lower case. Best way to start this process is to use the template provided rightly named *template.xlsx*.

For the purpose of this exercise, there is no clear definition of retirement age. There will be a year, however, from which you will stop having anticipated income, or diminished income due to decreasing your work load. This transition can be gradual or sudden. Therefore there is no need to enter a retirement age for the sole purpose of quantifying your financial future.

Contributions to your savings accounts are marked as *ctrb*. Contributions to your 401k must also include your employer's contributions. As this file is in Excel, one can use the native calculator to enter a percentage of the anticipated income for contributions as this can sometimes be easier.

Roth conversion are specified in the column marked *Roth X*. Roth conversion are typically performed in the years when the income is lower (and therefore lower tax rates), typically in the bridge years between having a full-time regular salary and collecting social security.

Finally, *big ticket items* are used for accounting for the sale or purchase of a house, or any other major expense or money that you would give or receive (e.g., inheritance, or large gifts to or from you). Therefore, the sign (+/-) of entries in this column is important. Positive numbers are deposits in individual taxable savings account, while negative numbers might involve withdrawals and distributions from retirement accounts. All other column entries should be positive.

The tab name for each spreadsheet represents the name of the spouse for reporting yearly transactions affecting the plan. In fact, Owl extracts the names of these tabs to determine the individuals' names. Therefore, you need to rename these tabs to reflect your personal names if you want to accurately represent your case.

Note that the file format from the (free) LibreOffice software can also be read, so you do not need to have an Excel license. However, you might have to install the `odfpy` package through `conda install odfpy` first, which might require the *conda-forge* channel to be added (`conda config --add channels conda-forge`).

Jack and Jill has provided their specific information in the file *jack+jill.xlsx*. This worksheet file needs to have two tabs, the first one named *Jack* and the second one named *Jill*, following the same order of the data provided before. Open this file in Excel and familiarize yourself with its contents.

In [ ]:
plan.readContributions('jack+jill.xlsx')

## What are the current and future assets allocations?
Each savings account can invest in 4 major classes of assets:
- Equity funds tracking the S&P 500 index;
- Bond assets tracking the Corporate bonds (Baa) index;
- Fixed-income securities represented by the performance of 10-year Treasury notes;
- Inflation-indexed securities tracking the urban Consumer Price Index (common assets).

The total of percentages in each class of assets for each savings account must add to 100%.

You are asked to provide assets allocation ratios for today, and ones for the time at the end of your life.
Values in between will be interpolated using a linear operator by default. This can be useful
if you want to shift assets allocation as you age. One can also select an *S* curve for transitioning
from the initial value to the final value. This is done by choosing *s-curve* instead of *linear* as the method for interpolation. *S-curve* take two additional parameters, the *center* of the change which defaults to 15 years from now, and the *width* which defaults to 5 years. These parameters will induce
a gradual change from the initial allocations now, starting to change in 10 years (15 - 5), and converging to final allocations in 20 years (15 + 5). 


In the example below, Jack's and Jill's assets allocations start in full stock equities in the tax-free account,
gradually transitioning to a more conservative portfolio towards the end of their life. Jill's tax-deferred
account stays as a traditional 60/40 for the duration of the realization. Note that assets allocation ratios are entered as percentages, and that these percentages are for each type of savings account, and for each spouse.

In [ ]:
plan.setInterpolationMethod('s-curve')
plan.setAllocationRatios(
    'account',
    taxable=[[[60, 40, 0, 0], [70, 30, 0, 0]], [[60, 40, 0, 0], [70, 30, 0, 0]]],
    taxDeferred=[[[60, 40, 0, 0], [70, 30, 0, 0]], [[60, 40, 0, 0], [70, 30, 0, 0]]],
    taxFree=[[[60, 40, 0, 0], [70, 30, 0, 0]], [[60, 40, 0, 0], [70, 30, 0, 0]]],
)

### Show assets allocations during the time period
The allocation of assets is shown for the three types of savings accounts (taxable, tax-deferred, and tax-free) for each spouse and for the 4 types of investments: stocks (S&P 500), corporate bonds (Baa), Treasury notes (10-y), and common assets tracking inflation.

In [ ]:
plan.showAllocations()

## What about anticipated fixed income?
Pension and social security are fixed income. Model here assumes that pension income is not inflation adjusted while social security benefits are (but Owl can easily be modified to account for inflation-adjusted pensions). Numbers to be provided are the predicted annual amount for each spouse and the age of the commencement of benefits.

By default, no pension benefits are assumed. This can also be specified explicitly by entering zeros (0) as entries, as in

    plan.setPension([0, 0], [65, 65])
    
For social security, one must provide the predicted annual amount(s) and the starting age(s) at which benefits are anticipated to be received. There are plenty of social security benefit estimators on the web, including the info you can get directly from your own account at the Social Security Administration (ssa.gov). Another interesting calculator can be found at www.opensocialsecurity.com. This calculator allows you to compare different scenarios regarding your commencement age through a sensitivity plot.


Here, Jill has an unindexed pension of \\$10 k per year. Both Jack and Jill believe they have good genes and decided to take their social security benefits at age 70. The amounts provided are estimation of the amounts they would receive at age 70. 

In [ ]:
plan.setPension([0, 10], [65, 65])
plan.setSocialSecurity([28, 25], [70, 70])

## How much net income is desirable at retirement?
For determining the desirable annual net income in retirement, certified planners will strongly suggest that you've must have already done a cash flow analysis on your yearly spending. After this exercise, you should have a good idea of how much you'll need in retirement. Another approach is to experiment with multiple spending scenarios and see what your current and future savings can sustain under different market conditions. As one nears retirement, both approaches will need to meet at one point.

The desired income defined here is the minimum annual **net** income (i.e., after paying federal income tax) that one would like to receive starting at her/his "retirement age" (we provided a loose definition of the term *retirement age* above). This desired income must be adjusted for inflation and can follow an additional adjustment called a *smile* profile. A *smile* profile accounts for the fact that your spending capacity will modulate during retirement as you go from the so-called gogo years to the no-go years. A *flat* profile, on the other hand, will keep the same value, which will only be adjusted for inflation. More realistically, the income could also be modulated by the performance of assets, reducing withdrawals in market down years. These strategies (first proposed by Guyton and Klinger) could be implemented in Owl. 

A second argument can also be provided to specify the fraction of income left the the survivor. The default is 60%.

The target and actual net income values achieved through the realization can be plotted as will see below.


Jack and Jill believe that their spending profile will follow a *smile* curve rather than a *flat* line.

In [ ]:
plan.setSpendingProfile('smile')
plan.showProfile()

## Specify rates of return and inflation rate
Rates of return for each class of assets can be specified, including the rate of inflation.
These are important components for  future assumptions.
Setting future rates is done with the `setRates()` method from which one can select different
sources for determining the rates.
Valid choices are *historical*, *stochastic*, *average*, *fixed*, *conservative*, or *default*.

For the *realistic* case, values reported from MorningStar for the next ten years are used. It is selected by using this call:

    plan.setRates('realistic')

*Conservative* rates are lower than the last ten years as most analysts are predicting lower rates from the next decade. These values are fixed rates of 6.0% for the S&P 500, 4.0% for the Corporate bonds markets, 3.5% for the 10-year Treasury notes, with an inflation at around 2.8%. Using these conservative rates can be achieved by using the following call:

    plan.setRates('conservative')

For the *historical*, *stochastic*, and *average* options, data from 1928 to the last year are available for experimenting.
Ranges chosen smaller than the life horizon of the longest-lived individual will have rate values repeated in cycle. For example,
choosing historical data from 1994 to (up and including) 1996 will repeat these three values over the time span of the realization.
This would be called as follows:

    plan.setRates('historical', 1994, 1996)

This case is only provided as an explanatory example, as it would have little practical value.

If the upper bound is not provided as the third argument, then the latest data year (i.e., last year) will be assumed by default.
If one chooses a historical range starting from 1970, Owl will use the rates of 1970 for this year and 1971 for next, etc. This would be simulated as follows:

    plan.setRates('historical', 1970)
  
Due to its particular sequence of rates, the worst-case historical scenario is a retirement starting in 1966. This can be simulated as follows:

    plan.setRates('historical', 1966)

In this case, the  current year will have the same rates as those that happened in 1966, and next year will have those from 1967, and so on. This choice is given for instructional purposes only. No one should make a plan based on the worst-case historical scenario. Nevertheless, it can be informative to test your own case. In practice, however, a success rate larger than 90% over a reasonable set of historical starting years and market assumptions would be acceptable by a large portion of rational thinkers. But this is all a personal choice. We'll cover how to model multiple starting years below.

Alternatively, one can choose a *stochastic* approach in which case the rates are determined from the multivariate distribution for the 4 rates in the selected year range.  The computed statistical distribution of the selected range of data is used to generate random rate values. For example,

    plan.setRates('stochastic', 1945)
    
will analyze the annual rates from 1945 up to last-year and compute means and covariance to generate new data that are statistically representative of the ones observed during this selected time period. The rates randomly generated for the time span can be plotted and examined as we will see below. Similarly,

    plan.setRates('stochastic', 1940, 1970)

would generate random rates consistent to those observed during the 1940 - 1970 time period.

Rates can also be set to fixed values obtained from an average over a time interval using the *average* option. For example, the call

    plan.setRates('average', 1990, 2020)

would set the rates to constant values being the average observed from 1990 to 2020 inclusively.
    
Finally one can also use fixed annual rates by providing a list of 4 entries in percent as follows:

    myrates = [9.6, 4.0, 3.0, 3.8]
    plan.setRates('fixed', values=myrates)
    
This example would use fixed rates of 9.6%, 4.0%, 3.0%, and 3.8% as average annual returns on S&P 500, corporate bonds, Treasury notes, and common assets, respectively, with an average annual inflation rate of 3.8% for the full duration of the time simulation. Recall that the common assets class consists of investments tracking inflation only. Therefore the last index serves both to track the common asset class and to adjust values for inflation.

Also note that the S&P 500 rates provided always include dividends, which are assumed to be reinvested.


Jill is interested to know if her situation would survive a retirement started in 1969. The following call would model exactly that situation.

In [ ]:
plan.setRates('historical', 1969)

#### Show annual rates used for calculations
As described above, there are many choices for selecting rates. This graph will display the annual rates used during the time span of this realization.

In [ ]:
plan.showRates()

##### Show historical rate distribution
Since Owl has the historical rates available, one can also display their histograms using a simple function call. This is done with `owl.showRateDistribution()`.

Given the standard deviation of each histogram, the risk/benefit between stocks and bonds is clear. As this function returns the means and covariance matrix, we assign the result to the `_` variable which, in Python, is used to store results from the last command. Let's look at the rates distribution over the 30-year period running from 1969 to 1999. Notice how high is the average inflation.

In [ ]:
owl.showRateDistributions(1969, 1999)

### Other configurable parameters
There are a few more parameters that can be configured. The following calls are provided as examples. If not specified, the default values are:
- Heirs marginal tax rate: 30%
- Tax rate on long-term capital gains: 15%
- Dividend return rate on equities: 2%
These are provided for reference.

In [ ]:
# plan.setHeirsTaxRate(30)
# plan.setLongTermIncomeTaxRate(15)
# plan.setDividendRate(2)

## Generating the outcome of a scenario
We're now ready to run a single instance of a scenario. 

### Maximum net income
Recall that we set the rates above to mimic those from and after 1969. For this purpose, we only need to run a single realization looking at what would happen if Jack and Jill had retired in 1969. So let's run this plan: The following function call runs all the required calculations for the time horizon over Jack and Jill's life expectancy. The function `solve()` will optimize the selected scenario using linear programming. But we first need to select what needs to be optimized by passing options.

Let's first maximize the net income subject to leaving a bequest of 500 k\\$ and limiting Roth conversions to be larger than 10k\\$ but less than 100k\\$ if needed. If `maxRothConversion` is set to 'file', only conversions in the contribution worksheet file will be performed.

In [ ]:
options = {'maxRothConversion': 100, 'estate': 500}
plan.solve('maxIncome', options=options)

#### Analysis

#### Show net income compared to target income over the years
This graph shows how the actual net income generated by the plan realization matches the inflation-adjusted net income profile specified. Note the 40% drop in target income as one spouse passes. This drop is configurable through the `setProfile()` method. Default value is 60%.

In [ ]:
plan.showNetIncome()

### Show taxable annual income and anticipated tax brackets
Gross income also includes Roth conversions and big-ticket items, both of which are not contributing to your net income. This graph shows Jack and Jill's gross taxable income and how it compare with some anticipated Federal tax marginal brackets. This visualization is very convenient when one wants to perform Roth conversions and remain below a certain tax bracket.

Note the shift in tax brackets taking place as the Tax Cut and Job Act expires after 2025.

In [ ]:
plan.showGrossIncome()

### Show annual taxes paid over the years of the realization
This graph shows how much Jack and Jill paid in federal taxes and IRMAA income-related Medicare insurance monthly adjustments over the years of this realization.

In [ ]:
plan.showTaxes()

#### Show sources of income over the years
Income will typically come from multiple sources, and it can be quite complex. This graphs shows the breakdown of Jack and Jill's sources of income by spouse and by origin. Note that distributions from tax-deferred accounts (*dist*) are distinguished from required minimum distributions (*rmd*) as they serve different purposes. Other labels should be self-explanatory.

In [ ]:
plan.showSources()

#### Show savings accounts balances at the beginning of each year
The balance for each savings account for each spouse is calculated at the beginning of each year. Another important aspect of this graph is how much is left at the end of the realization. This will be addressed in the next cell.

In [ ]:
plan.showAccounts()

To get more information about accounts at the last year of the realization, one can use

      plan.sumary()
    
which returns the value of the assets in today's \\$ at the last year of the scenario, assuming (in this case) a 30\% tax burden on the taxable portion of the estate (read tax-deferred savings accounts). The `summary()` function returns two values: the total post-tax value of all savings account in today's dollars and the cumulative inflation rate between today and the last day of the realization.

In [ ]:
plan.summary()

### Maximum bequest
Alternatively, one could optimize for maximum bequest subject to a net income of 100k\\$ per year.

In [ ]:
options = {'maxRothConversion': 100, 'netIncome': 100}
plan.solve('maxBequest', options=options)

#### Analysis

In [ ]:
plan.showNetIncome()

In [ ]:
plan.showGrossIncome()

In [ ]:
plan.showTaxes()

In [ ]:
plan.showSources()

In [ ]:
plan.showAccounts()

In [ ]:
plan.summary()

## Can we save this realization?
This instance of a future realization contains information on the distribution amounts, including the required minimum distribution that had to be performed under the given assumptions. This info can be saved in an excel workbook with one spreadsheet (tab) for each spouse. Worksheet will also contain annual rates, income, income taxes, and account balances. Second `True/False` argument controls if existing files get overwritten. Also remember that Windows will not allow the file to be overwritten while the file is being opened in Excel. In that case, the script will ask you to close the file and will retry to save.

Here, this call will create an excel workbook with one spreadsheet (tab) for Jack and one for Jill. 

Open the file in Excel to see what it looks like. 

In [ ]:
plan.saveWorkbook('jack+jill-1969', True)

This notebook is provided as a basic example on what you can do to assess the robustness and sensitivity of your retirement strategy. Go and explore!


Enjoy!
